In [1]:
# Titre : Analyse Technique - Récupération et Visualisation de Données
# Contenu : Script d'analyse technique amélioré

"""
# Installation des dépendances
%pip install yfinance plotly pandas-ta -q
"""

import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configuration du dossier de sortie
OUTPUT_DIR = "outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Dossier de sortie créé : {os.path.abspath(OUTPUT_DIR)}")

class StockAnalyzer:
    def __init__(self):
        self.tickers = {
            "US": {
                "AAPL": "Apple",
                "MSFT": "Microsoft", 
                "GOOGL": "Alphabet",
                "AMZN": "Amazon",
                "NVDA": "NVIDIA",
                "TSLA": "Tesla",
                "META": "Meta Platforms"
            },
            "EU": {
                "TTE.PA": "TotalEnergies",
                "AI.PA": "Air Liquide",
                "AIR.PA": "Airbus",
                "BNP.PA": "BNP Paribas",
                "MC.PA": "LVMH",
                "SAP.DE": "SAP",
                "ASML.AS": "ASML"
            }
        }
        self.data_cache = {}
        
    def fetch_data(self, ticker, market="US", period="6mo"):
        """Récupère les données depuis Yahoo Finance"""
        try:
            print(f"   Téléchargement de {ticker}...")
            
            # Pour Yahoo Finance, on garde le ticker tel quel
            df = yf.download(
                ticker,
                period=period,
                interval="1d",
                progress=False,
                auto_adjust=True
            )
            
            if df.empty:
                print(f"   Aucune donnée pour {ticker}")
                return None
            
            # Gestion des multi-index columns
            if isinstance(df.columns, pd.MultiIndex):
                # Prendre le premier niveau si c'est un multi-index
                df.columns = df.columns.get_level_values(0)
            
            # Vérifier et sélectionner les colonnes nécessaires
            available_cols = []
            for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
                if col in df.columns:
                    available_cols.append(col)
                else:
                    # Essayer d'autres noms possibles
                    col_lower = col.lower()
                    if col_lower in df.columns:
                        df = df.rename(columns={col_lower: col})
                        available_cols.append(col)
            
            if 'Close' not in df.columns:
                print(f"   Colonne 'Close' manquante pour {ticker}")
                return None
            
            # Sélectionner uniquement les colonnes disponibles
            df = df[available_cols]
            
            # Compléter les colonnes manquantes si nécessaire
            if 'Open' not in df.columns:
                df['Open'] = df['Close']
            if 'High' not in df.columns:
                df['High'] = df['Close']
            if 'Low' not in df.columns:
                df['Low'] = df['Close']
            if 'Volume' not in df.columns:
                df['Volume'] = 0
            
            print(f"   {ticker}: {len(df)} jours, Close: {df['Close'].iloc[-1]:.2f}")
            return df
            
        except Exception as e:
            print(f"   Erreur pour {ticker}: {str(e)}")
            return None
    
    def calculate_indicators(self, df):
        """Calcule les indicateurs techniques"""
        if df is None or df.empty or 'Close' not in df.columns:
            return df
        
        try:
            # Copie pour éviter les warnings
            df = df.copy()
            
            # Moyennes mobiles (seulement si assez de données)
            if len(df) >= 20:
                df['MA20'] = df['Close'].rolling(window=20, min_periods=1).mean()
            else:
                df['MA20'] = df['Close']
                
            if len(df) >= 50:
                df['MA50'] = df['Close'].rolling(window=50, min_periods=1).mean()
            else:
                df['MA50'] = df['Close']
                
            if len(df) >= 200:
                df['MA200'] = df['Close'].rolling(window=200, min_periods=1).mean()
            else:
                df['MA200'] = df['Close']
            
            # RSI
            if len(df) >= 14:
                delta = df['Close'].diff()
                gain = (delta.where(delta > 0, 0)).rolling(window=14, min_periods=1).mean()
                loss = (-delta.where(delta < 0, 0)).rolling(window=14, min_periods=1).mean()
                rs = gain / loss.replace(0, np.nan)
                df['RSI'] = 100 - (100 / (1 + rs))
                df['RSI'] = df['RSI'].fillna(50)  # Valeur neutre par défaut
            else:
                df['RSI'] = 50
            
            # Bandes de Bollinger
            if len(df) >= 20:
                df['BB_middle'] = df['Close'].rolling(window=20, min_periods=1).mean()
                bb_std = df['Close'].rolling(window=20, min_periods=1).std()
                df['BB_upper'] = df['BB_middle'] + (bb_std * 2)
                df['BB_lower'] = df['BB_middle'] - (bb_std * 2)
            else:
                df['BB_middle'] = df['Close']
                df['BB_upper'] = df['Close']
                df['BB_lower'] = df['Close']
            
            # MACD
            if len(df) >= 26:
                exp1 = df['Close'].ewm(span=12, adjust=False, min_periods=1).mean()
                exp2 = df['Close'].ewm(span=26, adjust=False, min_periods=1).mean()
                df['MACD'] = exp1 - exp2
                df['Signal'] = df['MACD'].ewm(span=9, adjust=False, min_periods=1).mean()
            else:
                df['MACD'] = 0
                df['Signal'] = 0
            
            return df
            
        except Exception as e:
            print(f"   Erreur calcul indicateurs: {e}")
            return df
    
    def create_dashboard(self, df, ticker, name, save_html=True):
        """Crée un dashboard interactif"""
        if df is None or df.empty:
            print(f"   Pas de données pour créer le dashboard {ticker}")
            return None
        
        try:
            # Prendre les 100 derniers jours ou moins si pas assez de données
            plot_data = df.tail(min(100, len(df)))
            
            fig = make_subplots(
                rows=4, cols=1,
                shared_xaxes=True,
                row_heights=[0.4, 0.2, 0.2, 0.2],
                vertical_spacing=0.05,
                subplot_titles=[
                    f"{name} ({ticker}) - Prix et Moyennes Mobiles",
                    "Volume",
                    "RSI (14 périodes)",
                    "MACD"
                ]
            )
            
            # 1. Prix et moyennes mobiles
            fig.add_trace(
                go.Candlestick(
                    x=plot_data.index,
                    open=plot_data['Open'],
                    high=plot_data['High'],
                    low=plot_data['Low'],
                    close=plot_data['Close'],
                    name="OHLC"
                ),
                row=1, col=1
            )
            
            # Ajouter les MAs seulement si elles existent
            if 'MA20' in plot_data.columns:
                fig.add_trace(
                    go.Scatter(x=plot_data.index, y=plot_data['MA20'], 
                              name="MA20", line=dict(color='orange', width=1)),
                    row=1, col=1
                )
            
            if 'MA50' in plot_data.columns:
                fig.add_trace(
                    go.Scatter(x=plot_data.index, y=plot_data['MA50'], 
                              name="MA50", line=dict(color='red', width=1)),
                    row=1, col=1
                )
            
            if 'MA200' in plot_data.columns:
                fig.add_trace(
                    go.Scatter(x=plot_data.index, y=plot_data['MA200'], 
                              name="MA200", line=dict(color='purple', width=1)),
                    row=1, col=1
                )
            
            # 2. Volume avec couleur
            if 'Volume' in plot_data.columns:
                colors = ['green' if plot_data['Close'].iloc[i] >= plot_data['Open'].iloc[i] else 'red' 
                         for i in range(len(plot_data))]
                
                fig.add_trace(
                    go.Bar(
                        x=plot_data.index,
                        y=plot_data['Volume'],
                        name="Volume",
                        marker_color=colors,
                        opacity=0.7
                    ),
                    row=2, col=1
                )
            
            # 3. RSI
            if 'RSI' in plot_data.columns:
                fig.add_trace(
                    go.Scatter(x=plot_data.index, y=plot_data['RSI'], 
                              name="RSI", line=dict(color='blue', width=2)),
                    row=3, col=1
                )
                
                fig.add_hline(y=70, line_dash="dash", line_color="red", row=3, col=1)
                fig.add_hline(y=30, line_dash="dash", line_color="green", row=3, col=1)
                fig.add_hline(y=50, line_dash="dot", line_color="gray", row=3, col=1)
            
            # 4. MACD
            if 'MACD' in plot_data.columns and 'Signal' in plot_data.columns:
                fig.add_trace(
                    go.Scatter(x=plot_data.index, y=plot_data['MACD'], 
                              name="MACD", line=dict(color='blue', width=2)),
                    row=4, col=1
                )
                
                fig.add_trace(
                    go.Scatter(x=plot_data.index, y=plot_data['Signal'], 
                              name="Signal", line=dict(color='red', width=1)),
                    row=4, col=1
                )
            
            # Mise en forme
            fig.update_layout(
                height=1000,
                showlegend=True,
                template="plotly_dark",
                xaxis_rangeslider_visible=False,
                title=f"Analyse Technique - {name} ({ticker})"
            )
            
            fig.update_xaxes(rangeslider_thickness=0.05)
            
            # Sauvegarde HTML
            if save_html:
                try:
                    filename = os.path.join(OUTPUT_DIR, f"{ticker.replace('.', '_')}_{datetime.now().strftime('%Y%m%d')}.html")
                    fig.write_html(filename)
                    print(f"   Dashboard sauvegardé: {filename}")
                except Exception as e:
                    print(f"   Erreur sauvegarde HTML: {e}")
            
            return fig
            
        except Exception as e:
            print(f"   Erreur création dashboard {ticker}: {e}")
            return None
    
    def generate_report(self, save_files=True):
        """Génère un rapport complet pour tous les tickers"""
        results = []
        dashboards = []
        
        print(" Début de l'analyse technique...")
        print("=" * 50)
        
        for market, tickers in self.tickers.items():
            print(f"\n Analyse du marché {market}:")
            print("-" * 30)
            
            for ticker, name in tickers.items():
                print(f"\n🔍 {name} ({ticker}):")
                
                # Récupération des données
                df = self.fetch_data(ticker, market, period="6mo")
                
                if df is not None and not df.empty:
                    # Calcul des indicateurs
                    df = self.calculate_indicators(df)
                    self.data_cache[ticker] = df
                    
                    # Statistiques
                    latest = df.iloc[-1]
                    prev_close = df['Close'].iloc[-2] if len(df) > 1 else latest['Close']
                    daily_change = ((latest['Close'] - prev_close) / prev_close * 100) if prev_close != 0 else 0
                    
                    stats = {
                        'Ticker': ticker,
                        'Nom': name,
                        'Marché': market,
                        'Dernier Close': round(latest['Close'], 2),
                        'Variation 1j (%)': round(daily_change, 2),
                        'MA20': round(latest['MA20'], 2) if 'MA20' in latest else None,
                        'MA50': round(latest['MA50'], 2) if 'MA50' in latest else None,
                        'RSI': round(latest['RSI'], 1) if 'RSI' in latest else None,
                        'Signal RSI': '🔴 Sur-achat' if ('RSI' in latest and latest['RSI'] > 70) 
                                    else '🟢 Sous-vente' if ('RSI' in latest and latest['RSI'] < 30) 
                                    else '⚪ Neutre',
                        'Volume Moyen (30j)': int(df['Volume'].tail(30).mean()) if len(df) >= 30 else int(df['Volume'].mean())
                    }
                    results.append(stats)
                    
                    # Création du dashboard
                    if save_files:
                        fig = self.create_dashboard(df, ticker, name, save_html=True)
                        if fig:
                            dashboards.append((ticker, fig))
                    
                    print(f"    Analyse terminée: {stats['Dernier Close']} € ({stats['Variation 1j (%)']}%) - RSI: {stats['RSI']}")
                else:
                    print(f"    Échec de l'analyse pour {ticker}")
        
        # Création du DataFrame de synthèse
        if results:
            summary_df = pd.DataFrame(results)
            
            # Sauvegarde CSV
            if save_files:
                try:
                    csv_filename = os.path.join(OUTPUT_DIR, f"summary_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv")
                    summary_df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
                    print(f"\n Rapport CSV sauvegardé: {csv_filename}")
                    
                    # Sauvegarde Excel avec mise en forme
                    excel_filename = os.path.join(OUTPUT_DIR, f"summary_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx")
                    with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
                        summary_df.to_excel(writer, sheet_name='Résumé', index=False)
                    print(f" Rapport Excel sauvegardé: {excel_filename}")
                    
                except Exception as e:
                    print(f" Erreur sauvegarde fichiers: {e}")
            
            print("\n" + "=" * 50)
            print(" Analyse terminée avec succès!")
            print(f" Fichiers sauvegardés dans: {os.path.abspath(OUTPUT_DIR)}")
            
            return summary_df, dashboards
        else:
            print("\n Aucune donnée récupérée!")
            return None, []

# Exécution principale
if __name__ == "__main__":
    # Initialisation de l'analyseur
    analyzer = StockAnalyzer()
    
    # Génération du rapport
    summary, dashboards = analyzer.generate_report(save_files=True)
    
    if summary is not None:
        # Affichage formaté dans le notebook
        print("\n RÉSUMÉ DES ANALYSES:")
        print("=" * 80)
        
        # Afficher le DataFrame avec style
        styled_summary = summary.style \
            .format({
                'Dernier Close': '{:.2f} €',
                'Variation 1j (%)': '{:.2f}%',
                'MA20': '{:.2f}',
                'MA50': '{:.2f}',
                'RSI': '{:.1f}'
            }) \
            .applymap(lambda x: 'background-color: #ffcccc' if isinstance(x, str) and 'Sur-achat' in x 
                     else 'background-color: #ccffcc' if isinstance(x, str) and 'Sous-vente' in x 
                     else '', subset=['Signal RSI']) \
            .bar(subset=['Variation 1j (%)'], align='mid', color=['#ff9999', '#99ff99'])
        
        display(styled_summary)
        
        # Statistiques globales
        print("\n STATISTIQUES GLOBALES:")
        print("-" * 40)
        if 'RSI' in summary.columns:
            print(f"RSI Moyen: {summary['RSI'].mean():.1f}")
            print(f"Actions en sur-achat: {len(summary[summary['Signal RSI'].str.contains('Sur-achat')])}")
            print(f"Actions en sous-vente: {len(summary[summary['Signal RSI'].str.contains('Sous-vente')])}")
        
        # Afficher quelques dashboards dans le notebook
        print("\n DASHBOARDS DISPONIBLES:")
        print("-" * 40)
        if dashboards:
            print(f"{len(dashboards)} dashboards générés")
            # Afficher le premier dashboard
            ticker, fig = dashboards[0]
            print(f"\nAffichage du dashboard pour {ticker}:")
            fig.show()
        else:
            print("Aucun dashboard généré")
        
        # Lien vers les fichiers
        print("\n FICHIERS GÉNÉRÉS:")
        print("-" * 40)
        if os.path.exists(OUTPUT_DIR):
            files = os.listdir(OUTPUT_DIR)
            for file in files:
                if file.endswith(('.html', '.csv', '.xlsx')):
                    filepath = os.path.join(OUTPUT_DIR, file)
                    print(f"• {file} ({os.path.getsize(filepath):,} octets)")

Dossier de sortie créé : c:\Users\gui\Projet github CV\Stock-Exchange-Analysis\Notebook\outputs
 Début de l'analyse technique...

 Analyse du marché US:
------------------------------

🔍 Apple (AAPL):
   Téléchargement de AAPL...
   AAPL: 128 jours, Close: 256.00
   Dashboard sauvegardé: outputs\AAPL_20260129.html
    Analyse terminée: 256.0 € (-0.17%) - RSI: 45.1

🔍 Microsoft (MSFT):
   Téléchargement de MSFT...
   MSFT: 128 jours, Close: 424.12
   Dashboard sauvegardé: outputs\MSFT_20260129.html
    Analyse terminée: 424.12 € (-11.94%) - RSI: 30.4

🔍 Alphabet (GOOGL):
   Téléchargement de GOOGL...
   GOOGL: 128 jours, Close: 331.36
   Dashboard sauvegardé: outputs\GOOGL_20260129.html
    Analyse terminée: 331.36 € (-1.38%) - RSI: 56.1

🔍 Amazon (AMZN):
   Téléchargement de AMZN...
   AMZN: 128 jours, Close: 239.18
   Dashboard sauvegardé: outputs\AMZN_20260129.html
    Analyse terminée: 239.18 € (-1.58%) - RSI: 41.7

🔍 NVIDIA (NVDA):
   Téléchargement de NVDA...
   NVDA: 128 jours, C

,Ticker,Nom,Marché,Dernier Close,Variation 1j (%),MA20,MA50,RSI,Signal RSI,Volume Moyen (30j)
0,AAPL,Apple,US,256.00 €,-0.17%,258.15,268.41,45.1,⚪ Neutre,45401703
1,MSFT,Microsoft,US,424.12 €,-11.94%,467.24,477.42,30.4,⚪ Neutre,26659416
2,GOOGL,Alphabet,US,331.36 €,-1.38%,327.27,316.77,56.1,⚪ Neutre,28774149
3,AMZN,Amazon,US,239.18 €,-1.58%,238.53,232.36,41.7,⚪ Neutre,38168436
4,NVDA,NVIDIA,US,188.38 €,-1.64%,186.28,183.64,54.7,⚪ Neutre,158137670
5,TSLA,Tesla,US,422.76 €,-2.02%,438.28,443.27,43.5,⚪ Neutre,66266025
6,META,Meta Platforms,US,728.32 €,8.91%,648.66,645.21,70.6,🔴 Sur-achat,15925395
7,TTE.PA,TotalEnergies,EU,60.82 €,1.37%,56.82,55.84,82.6,🔴 Sur-achat,3818814
8,AI.PA,Air Liquide,EU,158.18 €,0.82%,157.73,160.32,47.8,⚪ Neutre,717043
9,AIR.PA,Airbus,EU,194.00 €,-0.54%,208.97,202.20,21.1,🟢 Sous-vente,976189



 STATISTIQUES GLOBALES:
----------------------------------------
RSI Moyen: 47.5
Actions en sur-achat: 2
Actions en sous-vente: 3

 DASHBOARDS DISPONIBLES:
----------------------------------------
14 dashboards générés

Affichage du dashboard pour AAPL:



 FICHIERS GÉNÉRÉS:
----------------------------------------
• AAPL_20260121.html (4,883,365 octets)
• AAPL_20260129.html (4,883,374 octets)
• AIR_PA_20260121.html (4,883,269 octets)
• AIR_PA_20260129.html (4,883,228 octets)
• AI_PA_20260121.html (4,883,630 octets)
• AI_PA_20260129.html (4,883,557 octets)
• AMZN_20260121.html (4,883,452 octets)
• AMZN_20260129.html (4,883,430 octets)
• ASML_AS_20260121.html (4,883,209 octets)
• ASML_AS_20260129.html (4,883,206 octets)
• BNP_PA_20260121.html (4,883,477 octets)
• BNP_PA_20260129.html (4,883,447 octets)
• GOOGL_20260121.html (4,883,359 octets)
• GOOGL_20260129.html (4,883,359 octets)
• MC_PA_20260121.html (4,883,374 octets)
• MC_PA_20260129.html (4,883,377 octets)
• META_20260121.html (4,883,451 octets)
• META_20260129.html (4,883,406 octets)
• MSFT_20260121.html (4,883,887 octets)
• MSFT_20260129.html (4,883,780 octets)
• NVDA_20260121.html (4,883,573 octets)
• NVDA_20260129.html (4,883,519 octets)
• SAP_DE_20260121.html (4,883,434 octet